In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 2]
%matplotlib widget
import seaborn as sns

import sc3nb as scn
from pya import Asig

In [ ]:
# import sonecules
# %run prepare-data.ipynb
# sonecules.startup()

### ICAD 2023
# sonecules: A Python Sonfication Architecture
### Dennis Reinsch and Thomas Hermann
#### Ambient Intelligence Group, Bielefeld University, Bielefeld, Germany

# Sonification Process



<center>
<img src="./figures/ecosystem-transpose-process.drawio.svg" width=100%/>
</center>

# Aproaches to Sonification


<img src="figures/Sonecules-Motivation1.png" width="100%">

<img src="figures/Sonecules-Motivation2.png" width="100%">

Summary: 
There are these two extremes:

* (i) open sound synthesis platforms high-jacked for sonification

* (ii) all-in-one graphical sonification design programs

* both create a walled garden

two extrema
(i) open sound synthesis platforms high-jacked for sonification
- examples: SuperCollider, PureData, Max/MSP, Csound
- requires knowledge in regard of sound synthesis to even get started.
  something what seems strange when we relate it to visualization
- often lack data handling
(ii) complex all-in-one graphical sonifi- cation design programs: in this case complete/self-contained pack- ages are provided which often come with GUI, data import func- tions, and one (or few) very specific sonification design(s).
- examples: Highcharts Sonification Studio, Sonification Workstation, or Rotator
- Such systems are much more beginner-friendly as compared to the above class of approaches,
- large code base to integrate the implemented sonification design with specific data loading and processing capabilities and in addition provide a graphical user interface for all this.
- but they are quite limited to their specific use case as they are hard (if not impossible) to extend – or even to be adjusted in parameters, if developers did not already offer a control for it.


To our opinion, both of these extremes create some kind of walled garden which hinders the sonification community to share their methods and grow
We believe that the walls can be teared down by following the example of how computer graphics was made available to users,

# Toolchain Solution

<center>
<img src="./figures/ecosystem-transpose.drawio.svg" width=100%/>
</center>

* do one thing but do it well - UNIX principle

## Sonecules: Getting Started

In [ ]:
import sonecules

Now we startup, which uses the default backend (sc3/ resp. sc3nb).

In [ ]:
sonecules.startup()

* But other backends such as pya are available...

Next let's load some data (as pandas Dataframes) to demo our Sonecules

In [ ]:
%run prepare-data.ipynb

* Using mesonic, Sonecules place sonification events in a timeline
* A Timeline is played via a playback
* Rendering can be realtime or in non-realtime
* Let's get our playback to start & stop it

### Fundamental Concepts: Context

In [ ]:
context = sonecules.gcc()

context.reset()                     # empty timeline (aka clear figure)

# create a synth
s1 = context.synths.create("s1", mutable=False)    

# schedule sonic marks
with context.at(0.4): s1.start(freq=400, dur=1.6)
with context.at(0.6): s1.start(freq=500, dur=1.6)
with context.at(0.8): s1.start(freq=600, dur=1.6)

context.timeline.plot()

### Fundamental Concept: Playback and Timeline (from mesonic)

to play the sonification we need a playback

In [ ]:
playback = sonecules.playback()

This can be start(), stop() and we can check and set the time

In [ ]:
playback.start()  

In [ ]:
playback.time

In [ ]:
playback.time = 0.6

In [ ]:
playback.stop()

NOTE: Dennis->Thomas

# Sonecules Demonstration

* Now we use Sonecules, i.e. capsuled sonification classes
* which give a more high-level access to sonification methods
* Let's look at 
  * Audification, 
  * Buffer-based Mappings: Data modulating sound parameters at high frequencies
  * Score-based methods: special versions for Continuous Mappings
  * Classical Parameter Mapping Sonifications
    * continuous Parameter Mapping Sonification
    * discrete Parameter Mapping Sonification
  * Model-based Sonification: Data Sonogram
  * Earcons

In [ ]:
import sonecules
sonecules.pb = sonecules.playback
context = sonecules.gcc()

### Audification Sonecule

In [ ]:
from sonecules.buffersyn import Audification

Let's test with EEG data (of an epileptic seizure).
Here is the data

In [ ]:
dasig = Asig(eeg_data, sr=250)
plt.figure(); dasig.plot(offset=1, color='r', lw=0.5);

In [ ]:
audification = Audification(data=eeg_data[:,[0,8]], sr=256)

In [ ]:
context.reset()
audification.schedule(0, params={"rate": 10})
sonecules.pb().start()

You can check more parameters of the used synth:

In [ ]:
audification.synth.params

In [ ]:
context.reset()
audification.schedule(0, params={"rate": 20, "amp": 0.5, "pan": -1})
sonecules.pb().start()

## Score-based mappings

Let's load some data for the example

In [ ]:
# take segment at onset of epilepsy, only selected channels, decimate by 5
data = Asig(eeg_data, sr=250)[{7.5:10.5}, [0,1,2,5,9,12]][::5]
plt.figure(); data.plot(offset=2);

### Time Variant Oscillator

In [ ]:
from sonecules.scoresyn import TVOSon

snctvo = TVOSon(dasig[{7.5: 10.5}, [1, 2, 3]][::2])

context.reset()
snctvo.schedule(at=0, rate=1,
    base_pitch=60,
    pitch_step=12,
    pitch_relwid=0.1,
    amp_mode="change",
    level=-10,
    map_mode="channelwise",
)
sonecules.pb().start(rate=1)

### Special Case: Polyphonic Sonification

In [ ]:
snctvo = TVOSon(dasig[{7.5: 11.5}, [0,1,2,3,4,5]][::2])
context.reset()
snctvo.schedule(at=0, rate=0.5,
    base_pitch=50,
    pitch_step=[0,4,7,12,16,19],  # selected musical tones for channels
    pitch_relwid=0,  # try 0.5 
    amp_mode="change",
    level=-5,
    map_mode="channelwise",
)
sonecules.pb().start(rate=1)

### Special Case: Timbral Sonification (and more)

In [ ]:
# timbral sonification is just a special case of TVOSon
snctvo = TVOSon(dasig[{7.5: 10.5}, :][::2])

f0 = 60
base_pitch = scn.cpsmidi(f0)
pitch_steps = [ scn.cpsmidi(f0*(i+1)) - base_pitch for i in range(dasig.channels)] 

context.reset()
snctvo.schedule(at=0, rate=0.3,
    base_pitch=base_pitch,
    pitch_step=pitch_steps,
    pitch_relwid=0,  # use 1.5 for pitch added effect
    amp_mode="change",
    level=-10,
    map_mode="channelwise",
)
sonecules.pb().start(rate=1)

In [ ]:
sonecules.stop()

This special case works via a score: 
- each data involves a set event on a synth (sent via OSC)
- but alternatively mapping could also be managed by buffers
- then each buffer directly modulates the amplitude of an oscillator
- For that we can already show a special Sonecule:

### Timbral-Sonification (as own sonecule via buffers)

In [ ]:
timbralson = sonecules.buffersyn.TimbralSon(eeg_data[14*256:18*256], sr=256)

context.reset()
timbralson.schedule(at=0.5, params={"amp": 0.05, "f0": 120, "rate": 0.5})
sonecules.pb().start(rate=1)

- this scales better with high-frequency data (e.g. >1000 changes/second)
- a score-based approach would result in issues such as
  - python cpu bottleneck
  - non-accurate (due to control rate being lower than audio rate)
- but this depends on the backend 

In [ ]:
sonecules.stop()

## Classical Parameter Mapping Sonification

There are several ways of doing classical parameter mapping (both discrete and continuous)

In [ ]:
from sonecules.scoresyn import StandardContinuousPMSon, StandardDiscretePMSon
from sc3nb import midicps, linlin

Let's work with the penguins data set (measures of penguins)

In [ ]:
penguins_df

In [ ]:
# adapting linlin to take dmin dmax as params - names open for discussion
dlinlin = lambda value, dmin, dmax, y1, y2: linlin(value, x1=dmin, x2=dmax, y1=y1, y2=y2)

In [ ]:
# penguins_df[['body_mass_g', 'flipper_length_mm']).plot()
sns.pairplot(data=penguins_df, hue="species", x_vars='body_mass_g', y_vars='flipper_length_mm')

In [ ]:
dlinlin = lambda value, dmin, dmax, y1, y2: linlin(value, x1=dmin, x2=dmax, y1=y1, y2=y2)

scpmson = StandardContinuousPMSon("s2", 
    {"freq": {"bounds": (midicps(30), midicps(90))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1}}
)

test_mapping = {
    "onset": ("body_mass_g", dlinlin, {"y1": 0, "y2": 3}),
    "freq" : ("flipper_length_mm", dlinlin, {"y1": midicps(50), "y2": midicps(70)})
}

context.reset() # sn.reset() 

scpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)
# Error message when bounds are not respected by the mapping are currently quite hard to understand
# should fix this in mesonic 

sonecules.playback().start()

In [ ]:
sdpmson = StandardDiscretePMSon("s1", 
    {"freq": {"bounds": (midicps(49.9), midicps(70.1))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1}}
)  # bounds are in pre conversion unit - but this is done differently here in the code

test_mapping = {
    "onset": ("body_mass_g", dlinlin, {"y1": 0, "y2": 3}),
    "freq" : ("flipper_length_mm", dlinlin, {"y1": midicps(70), "y2": midicps(50)})
}

context.reset() # needed for now as sonecule.reset() does not work yet

sdpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)
sonecules.playback().start()


In [ ]:
context.timeline

### Continuous Parameter Mapping with specifying a callback function

In [ ]:
from sonecules.scoresyn import CPMSonCB

In [ ]:
plt.figure(); Asig(bld_df.iloc[:24*7, 8:].values, sr=24).plot(offset=1); plt.grid(); plt.title("Building Dataset");

In [ ]:
scb = CPMSonCB(bld_df.iloc[:7*24, 6:])  # one week
# scb = CPMSonCB(bld_df.iloc[:4000, 7:])  # whole dataset

mapcol = scb.mapcol

def callback_fn(r, cmi, cma, pp):
    pp['freq']      = scn.midicps(mapcol(r, 'temperature', cmi, cma, 48, 72))
    pp['amp']       = scn.dbamp(mapcol(r, 'humidity', cmi, cma, -20, 0)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['numharm']   = mapcol(r, 'solar_radiation', cmi, cma, 1, 12)
    pp['vibfreq']   = scn.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibintrel'] = 0
    return pp

context.reset()
scb.schedule(at=0, duration=4, callback_fn=callback_fn)
sonecules.playback().start()

In [ ]:
scb.create_callback_template(auto_assign=True)

In [ ]:
def cbfn(r, cmi, cma, pp):
    # columns are:'sunday' 'hour_from_noon' 'hour' 'am_pm' 
    # 'temperature' 'humidity' 'solar_radiation' 'wind_speed' 
    # 'hc_wb_electrical' 'hc_wb_cold_water' 'hc_wb_hot_water' 
    pp['freq']     	 = mapcol(r, 'sunday', cmi, cma, 123.48568035539805, 246.9713607107961)
    pp['amp']      	 = mapcol(r, 'hour_from_noon', cmi, cma, 0.41876930734447715, 0.8375386146889543)
    pp['vibintrel']	 = 0 
    pp['numharm']  	 = mapcol(r, 'solar_radiation', cmi, cma, 1, 5)
    pp['pint']     	 = 0
    pp['pan']      	 = mapcol(r, 'hc_wb_electrical', cmi, cma, -0.3400964812712826, -0.6801929625425652)

# create sonification e.g. by using
context.reset()
scb.schedule(at=0, duration=5, callback_fn=callback_fn)
sonecules.playback().start(rate=1)

NOTE: Thomas->Dennis

### Trigger Sonifications

TODO! Earcons - Event-based Sonification of mathematical function (TH)

### Model-Based Sonification

In [ ]:
from sonecules.triggersyn import DataSonogram

In [ ]:
sonecules.reset()
sonecules.gcc().enable_realtime()

In [ ]:
dsg2 = DataSonogram(penguins_df, x="flipper_length_mm", y="bill_length_mm", label="species")

## Roadmap for Demos of Sonifications for the ICAD paper

coarse line of thought for unfolding sonecules: from ANALOGIC -> MAPPING -> MODELS -> SYMBOLS (Earcons, Parameterized Auditory Icons)
 
Buffer
- Audification (BUFFER HOLDS DATA)
  - schedule
- Interactive Audification --> Widgets
  - create --> macht GUI
- Data Modulated Continuous Mappings: (HF Control Data->Parameters)
- Continuous Mapping using Buffers (audification near)
- Timbral Sonification (using Buffers) 
- TVOSC

Score
- TVOSC  --> dabei neu: MUTABLE
  - Graphical interface?
- CPMSon --> 1 synth, multiple parameter 
  - (freq, amp, pan, vibr (F/I), numharm, pulse rate)
- Classical good old DPMSon
- iris/penguin/buildung/glass

Event
- Event with all data lookahead (condition function)
- Event bekommt Stream und processed on the run
- Model-based Sonification



# Goals

## flexibility and portability
  
* build with Python
* allows adding new backends
* platform- and backend independent


## availability and distribution

* sonifications as reusable units
* growing collection

## reproducibility

* shared as actual sonification not as sound file
* experienceable with different data and parameters

## benchmarking

* enables comparisons between sonification designs
* enchancing scientific standards and development

## dissemination

* spreading sonifications as quickly usable tools
* reaching and enabling people without sonification expertise

# Conclusion

# ---


# Sonification Pipeline - All in One

<center>
<img src="./figures/soni-pipeline-allinone-small.jpg"/>
</center>

IN: The first example are tools that offer the user to handle the complete pipeline

However the resulting tools are often very specialized towards certain tasks and often can be regarded  as a graphical user interface for a single sonification design.

This does not allow the user to create completely new sonifications as the tools are not flexible enough, but rather use a certain sonification without much effort.


OUT: This is why many sonification experts prefer to built their own custom tools

# Sonification Pipeline - Combination of Tools

<center>
<img src="./figures/soni-pipeline-combi-small.jpg"/>
</center>

* sc3nb = Python + SuperCollder in Jupyter notebooks

IN: A common approach here is to use a sound synthesis engine like SuperCollider for creating the sound 

the data is prepared beforehand with other tools like Python.


While this approach offers the user a more flexible way it still requires the user to have knowledge about used sound synthesis engine


additionally it often becomes complicated to mix and match the different tools with each other and share the data



To tackle this problem we already created sc3nb which makes SuperCollider accessible from Python and allows the combined usage in one interactive Jupyter notebook

However sc3nb is strictly tied to SuperCollider and requires that the user is familiar with SuperCollider 


OUT: 

# mesonic

<center>
<img src="./figures/mesonic-pipeline.jpg"/>
</center>

IN: mesonic wants to go further by more directly focusing on the transformation.

this is done by providing a meso level that should be more flexible than high level (all in one) approaches 

but it should still offer the user an a shortcut to create new sonifications designs

instead of requiring the creation of custom tools, which are often hard to share and maintain

and require the user to deeply dive down into sound synthesis engines.

While at least some knowledge about sound synthesis is required to create a sonification it should be noted that the
most sound synthesis tools are quite complex and can be intimidating for new users that want create a new sonification

Additionally it is important to note that most sound synthesis engines are not specifically crafted for sonifications and often lack data processing capabilities.

OUT: The idea to introduce a layer in between the low level sound synthesis world and the high level sonification applications is inspired by the visualization domain


# matplotlib

<center>
<img src="./figures/anatomy-of-figure.jpg"/>
</center>

IN: If we look at matplotlib f.e. it becomes obvious that .. 

the plots does consists of different objects

These objects together form the complete plot 

As a user I do not need to create every single linie
but i can create complex plots by single function calls that will provide me all the single objects

Additionally it is possible to fine tune the plot using a object oriented approach.
and adjust f.e. the limits and the title.

This is what makes matplotlib easy to use but still a flexible basis for many other applications.

OUT: mesonic tries to adapt this idea to form a sonification from single parts and other ideas from the visualization domain into the domain of sonification



# mesonic - audio objects

<center>
<img src="./figures/mesonic-synth.jpg"/>
</center>

* practical use: discrete vs continuous Parameter Mapping Sonification

* inspired by Enge et al. 2021 - **0D / 1D auditory mark**

# mesonic - audio objects

<center>
<img src="./figures/mesonic-buffer-record.jpg"/>
</center>

# Example

In [ ]:
import mesonic
import numpy as np
from pya import *
import matplotlib.pyplot as plt
import matplotlib as mpl
from sc3nb import linlin, midicps, cpsmidi
import sc3nb as scn

Lets start by preparing mesonic

In [ ]:
context = mesonic.create_context()
context.enable_realtime();
context.processor.latency = 0.05
context.processor.latency

In this example we will use the EEG data from the [Supplementary material for "sc3nb: a Python-SuperCollider Interface for Auditory Data Science"](https://doi.org/10.4119/unibi/2956379)

In [ ]:
data = np.loadtxt("./files/epileptic-eeg.csv", delimiter=",")

We can simply create a stereo Buffer using this data

In [ ]:
buf = context.buffers.from_data(data[:,[0,1]], sr=256)
buf

And create a default Synth to play it back

In [ ]:
bsyn = context.synths.from_buffer(buf)

In [ ]:
bsyn # to see the synth's controls

let's audify the data in a loop

In [ ]:
bsyn.start(rate=20, amp=0.1, loop=1)

In [ ]:
bsyn.rate = 5
bsyn.amp = 1

In [ ]:
bsyn.stop()

A more advanced example using Granular Synthesis for interactive scrubbing of the buffer

In [ ]:
context.synths.buffer_synthdefs["tgrains"]= r"""
{ | bufnum={{BUFNUM}}, amp=0.3, rate=10, trate=5, pos=0 |
    var dur, cpos, sig;
    dur = 4 / trate;
    cpos = pos * BufDur.kr(bufnum);
    sig = TGrains.ar(2, Impulse.ar(trate), bufnum, rate, cpos, dur, 0, 0.5, 2);
    Out.ar(0, sig * amp);
}"""

In [ ]:
buf = context.buffers.from_data(data[:,5], sr=256)

In [ ]:
tgsyn = context.synths.from_buffer(buf, synth_name="tgrains")
tgsyn

In [ ]:
tgsyn.start(rate=20)

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(figsize=(8,2))
asig = Asig(data[:,5], sr=256).plot()

def on_move(event):
    if event.inaxes and event.button is mpl.backend_bases.MouseButton.LEFT:
        tgsyn.rate =  20 if event.ydata > 0 else 50
        tgsyn.pos = linlin(event.xdata, 0, 50, 0, 1)

binding_id = fig.canvas.mpl_connect('motion_notify_event', on_move)

In [ ]:
tgsyn.stop()

# mesonic concepts

<center>
<img src="./figures/mesonic-concepts.jpg"/>
</center>

Here you can see an overview of the concepts used in mesonic

- the Context can be regarded as counterpart of the Figure from matplotlib and as the central interface
    - it controls the backend which is clearly separated and designed to be exchangeable

- The Backend provides different Managers to create the Audio Objects
- and is also responsible for creating the Audio Object EventHandlers that will create sound in the backend 


- The Audio Objects  are the available building blocks for the sonification which are inspired by common concepts from sound synthesis software
- The different objects offer actions like f.e. starting a Synth with a certain frequency
These however do not directly generate sounds but rather create Events 

The events are then passed to the Context which
provides the sonification time for the event.
The Event then will be inserted into the timeline as a bundles 

The Timeline then forms a data structure that contains all the actions from the sonification

To actually listen to the sonification the events in the timeline can be rendered offline or by the playback

The playback offers an interactive control over the sonification and allows f.e. filtering by data source via the BundleProcessor
before passing the events to the Audio Object EventHandler in the Backend 



In [ ]:
print(context.synths.buffer_synthdefs["playbuf"])

In [ ]:
context.timeline.plot()